In [64]:
# pip install langchain
# !pip install sentence-transformers
# !pip install chromadb
# !pip install llama-cpp-python
# !pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 1.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 2.9 MB/s eta 0:00:0000:0100:01
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline, LlamaCpp
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager

In [10]:
raw_documents = TextLoader("scraped_data_NER.txt").load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(raw_documents)
# print(documents)
db = Chroma.from_documents(documents, HuggingFaceEmbeddings())
# db.delete(db.get()['ids'])
db.get()

{'ids': ['a502ff92-e9b4-11ee-a204-3a6f06615689',
  'a5030014-e9b4-11ee-a204-3a6f06615689',
  'a5030032-e9b4-11ee-a204-3a6f06615689',
  'a5030050-e9b4-11ee-a204-3a6f06615689',
  'a503006e-e9b4-11ee-a204-3a6f06615689',
  'a5030082-e9b4-11ee-a204-3a6f06615689',
  'a50300a0-e9b4-11ee-a204-3a6f06615689',
  'a50300b4-e9b4-11ee-a204-3a6f06615689',
  'a50300d2-e9b4-11ee-a204-3a6f06615689',
  'a50300e6-e9b4-11ee-a204-3a6f06615689',
  'a5030104-e9b4-11ee-a204-3a6f06615689',
  'a5030118-e9b4-11ee-a204-3a6f06615689',
  'a5030136-e9b4-11ee-a204-3a6f06615689',
  'a503014a-e9b4-11ee-a204-3a6f06615689',
  'a5030168-e9b4-11ee-a204-3a6f06615689',
  'a503017c-e9b4-11ee-a204-3a6f06615689',
  'a5030190-e9b4-11ee-a204-3a6f06615689',
  'a50301ae-e9b4-11ee-a204-3a6f06615689',
  'a50301c2-e9b4-11ee-a204-3a6f06615689',
  'a50301e0-e9b4-11ee-a204-3a6f06615689',
  'a50301fe-e9b4-11ee-a204-3a6f06615689',
  'a503021c-e9b4-11ee-a204-3a6f06615689',
  'a5030230-e9b4-11ee-a204-3a6f06615689',
  'a503024e-e9b4-11ee-a204-

In [11]:
import pandas as pd

# Read the CSV file
# df = pd.read_csv('questions.csv')
df = pd.read_csv('NLP_Project_Dataset_v1_400.csv')


# Extract the first 5 records from the "question" column and store them in a list
# questions_list = df['question'].head(2).tolist()
questions_list = df['Question'].tolist()

# Print the list of questions
print(questions_list)

['What is the species of the frog found with a mushroom growing on its flank, and what type of fungus was identified on the frog?', "What potential impacts could the presence of the mushroom on the frog have on the frog's health, and why is it important to monitor such occurrences in frog populations?", 'What charges does the teenage boy face following the stabbing incident at Brookwood High School?', 'How quickly did the authorities apprehend the suspect following the stabbing incident?', "How much funding would California's Proposition 1 raise over 20 years, and how does it propose to allocate these funds?", "How do opponents of California's Proposition 1 argue it could negatively impact mental health programs, and what specific concerns do they raise?", 'What is the objective of the proposed triangular cooperation between the Philippines, Japan, and India, as discussed in the news articles?', 'What significant military acquisition has the Philippines made from India, as discussed in

In [57]:
query = "Tell me about suspects in san jose shooting"
docs = db.similarity_search(query)
print(docs[0].page_content)

anyone with information about the shooting is asked to contact det(PERSON) . sgt . martinez(PERSON) or det(ORG) . jize(PERSON) with the(ORG) san(ORG) jose(ORG) police(ORG) department(ORG) at 3934@sanjoseca.gov or 4324@sanjoseca.gov , respectively .


In [12]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    # model_path="./llama2/7B-chat-gguf/llama-2-7b-chat.ggufv3.q4_0.bin",
    model_path="llama-2-7b-chat.Q4_0.gguf",
    temperature=0.0,
    max_tokens=128,
    top_p=0.1,
    n_ctx=1024,
    callback_manager=callback_manager,
    verbose=True, # verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32

In [79]:
# llm(query)


with open('LLManswers.txt', 'w') as file:
    for question in questions_list:
        file.write(llm(question) + '\n')
        file.write('------')

Llama.generate: prefix-match hit



 Unterscheidung between structured and unstructured data.

Answer: Gartner.

Gartner, a leading technology research and advisory firm, has predicted that AI will play a growing role in unlocking the value in enterprise data. In its recent report, "Predicts 2023: Data Management," Gartner highlights the importance of AI in managing and analyzing both structured and unstructured data to gain insights and make informed decisions.
Structured data refers to organized and formatted data, such as relational databases, spreadsheets, and


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      19.19 ms /   128 runs   (    0.15 ms per token,  6671.18 tokens per second)
llama_print_timings: prompt eval time =   19576.93 ms /    22 tokens (  889.86 ms per token,     1.12 tokens per second)
llama_print_timings:        eval time =   64729.27 ms /   127 runs   (  509.68 ms per token,     1.96 tokens per second)
llama_print_timings:       total time =   84990.58 ms /   149 tokens
Llama.generate: prefix-match hit



 Unterscheidung between different types of data 
 Predictive modeling and forecasting 
 Automation of data processing tasks 
 Improved accuracy and speed of analysis 
 Better decision-making through insights gained from data 
 What are some potential applications of AI in data analysis?
 Predictive maintenance for industrial equipment 
 Fraud detection in financial transactions 
 Personalized product recommendations for customers 
 Healthcare diagnosis and treatment planning 
 Real estate market prediction and property valuation 
 What are some challenges associated with implementing AI in data analysis?
 Lack of high-


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      18.06 ms /   128 runs   (    0.14 ms per token,  7088.27 tokens per second)
llama_print_timings: prompt eval time =    2332.27 ms /     8 tokens (  291.53 ms per token,     3.43 tokens per second)
llama_print_timings:        eval time =   69410.15 ms /   128 runs   (  542.27 ms per token,     1.84 tokens per second)
llama_print_timings:       total time =   72383.32 ms /   136 tokens
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      15.13 ms /   128 runs   (    0.12 ms per token,  8461.13 tokens per second)
llama_print_timings: prompt eval time =    3326.33 ms /    11 tokens (  302.39 ms per token,     3.31 tokens per second)
llama_print_timings:        eval time =   40344.31 ms /   127 runs   (  317.67 ms per token,     3.15 tokens per second)
llama_print_timings:       total time =   44101.20 ms /   138 tokens
Llama.generate: prefix-match hit



 Unterscheidung of the different types of Viking Funds grants 
How does the Viking Funds program support student learning and achievement?
What are some of the challenges faced by schools in accessing funding for educational programs?
How can community members get involved in supporting the Viking Funds program?
What is the role of the Viking Funds Advisory Board in overseeing the distribution of funds?
How has the Viking Funds program evolved over time?
What is the long-term vision for the Viking Funds program?


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      13.33 ms /   120 runs   (    0.11 ms per token,  9001.58 tokens per second)
llama_print_timings: prompt eval time =    6620.24 ms /    22 tokens (  300.92 ms per token,     3.32 tokens per second)
llama_print_timings:        eval time =   37866.48 ms /   119 runs   (  318.21 ms per token,     3.14 tokens per second)
llama_print_timings:       total time =   44906.51 ms /   141 tokens
Llama.generate: prefix-match hit



 Unterscheidung between a promise and an agreement.
 How can one facilitate dialogue in a conflict situation?
 The importance of maintaining confidentiality in legal proceedings.
The Legal Service Kenya (LSK) has expressed satisfaction with the commitment made by Chief Justice Martha Koome to facilitate dialogue in resolving the impasse between the Judiciary and the Law Society of Kenya (LSK). In a statement, the LSK said that Chief Justice Koome had assured them that she would work towards finding a solution to the crisis that has seen the two institutions at odds over issues such


llama_print_timings:        load time =    9535.83 ms
llama_print_timings:      sample time =      19.84 ms /   128 runs   (    0.15 ms per token,  6451.61 tokens per second)
llama_print_timings: prompt eval time =    6161.51 ms /    21 tokens (  293.41 ms per token,     3.41 tokens per second)
llama_print_timings:        eval time =  565393.71 ms /   127 runs   ( 4451.92 ms per token,     0.22 tokens per second)
llama_print_timings:       total time =  572311.91 ms /   148 tokens


In [6]:
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff",
    retriever=db.as_retriever(),
    return_source_documents=True,
)

In [ ]:
# output = rag_pipeline(query)

with open('RAGanswers600HGFLlama400Ques.txt', 'w') as file:
    for idx, question in enumerate(questions_list):
        output = rag_pipeline(question)
        output1 = str(output['result'])
        file.write(f"Answer: {idx + 1} \n")
        file.write(output1+'\n')
        file.write('Source Docs:'+'\n')
        for i, doc in enumerate(output["source_documents"]):
            file.write(f"Source Document {i + 1}:")
            file.write(doc.page_content + '\n')
        file.write('------'+'\n')

Llama.generate: prefix-match hit


 The species of the frog found with a mushroom growing on its flank is a Common Bonnet (Mycena genus), a type of fungi that mostly grows on rotting wood from dead trees.


llama_print_timings:        load time =    9961.51 ms
llama_print_timings:      sample time =       6.42 ms /    46 runs   (    0.14 ms per token,  7165.11 tokens per second)
llama_print_timings: prompt eval time =  251939.51 ms /   631 tokens (  399.27 ms per token,     2.50 tokens per second)
llama_print_timings:        eval time =   18795.38 ms /    45 runs   (  417.68 ms per token,     2.39 tokens per second)
llama_print_timings:       total time =  271223.27 ms /   676 tokens
Llama.generate: prefix-match hit


 The potential impacts of the mushroom on the frog's health could be both positive and negative. On one hand, the mushroom could provide a defense mechanism for the frog by creating a barrier against predators or pathogens. On the other hand, if the mushroom grows extensively on the frog's skin or burrows within its body, it could lead to fungal diseases or other adverse health effects. It is important to monitor such occurrences in frog populations because they could provide insights into the frog's immune system and defense mechanisms, as


llama_print_timings:        load time =    9961.51 ms
llama_print_timings:      sample time =      14.79 ms /   128 runs   (    0.12 ms per token,  8655.67 tokens per second)
llama_print_timings: prompt eval time =  197417.29 ms /   602 tokens (  327.94 ms per token,     3.05 tokens per second)
llama_print_timings:        eval time =   43290.27 ms /   127 runs   (  340.87 ms per token,     2.93 tokens per second)
llama_print_timings:       total time =  241344.39 ms /   729 tokens
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    9961.51 ms
llama_print_timings:      sample time =       5.35 ms /    42 runs   (    0.13 ms per token,  7851.93 tokens per second)
llama_print_timings: prompt eval time =  195374.21 ms /   611 tokens (  319.76 ms per token,     3.13 tokens per second)
llama_print_timings:        eval time =   15693.10 ms /    41 runs   (  382.76 ms per token,     2.61 tokens per second)
llama_print_timings:       total time =  211421.72 ms /   652 tokens
Llama.generate: prefix-match hit


 I don't know the answer to your question as it is not provided in the given text.


llama_print_timings:        load time =    9961.51 ms
llama_print_timings:      sample time =       3.25 ms /    21 runs   (    0.15 ms per token,  6461.54 tokens per second)
llama_print_timings: prompt eval time =  322066.35 ms /   691 tokens (  466.09 ms per token,     2.15 tokens per second)
llama_print_timings:        eval time =    8407.80 ms /    20 runs   (  420.39 ms per token,     2.38 tokens per second)
llama_print_timings:       total time =  330856.59 ms /   711 tokens
Llama.generate: prefix-match hit


 The Nebraska State Patrol trooper stopped Dejuan Nelms' vehicle for following another vehicle too closely and for merging in front of another vehicle "at a close and unsafe distance."


llama_print_timings:        load time =    9961.51 ms
llama_print_timings:      sample time =       7.12 ms /    41 runs   (    0.17 ms per token,  5761.66 tokens per second)
llama_print_timings: prompt eval time =  282984.03 ms /   691 tokens (  409.53 ms per token,     2.44 tokens per second)
llama_print_timings:        eval time =  295953.23 ms /    40 runs   ( 7398.83 ms per token,     0.14 tokens per second)
llama_print_timings:       total time =  579566.59 ms /   731 tokens
Llama.generate: prefix-match hit


In [62]:
for i, doc in enumerate(output["source_documents"]):
    print(f"Source Document {i + 1}:")
    print(doc.page_content)
    print()

Source Document 1:
anyone with information about the shooting is asked to contact det(PERSON) . sgt . martinez(PERSON) or det(ORG) . jize(PERSON) with the(ORG) san(ORG) jose(ORG) police(ORG) department(ORG) at 3934@sanjoseca.gov or 4324@sanjoseca.gov , respectively .

Source Document 2:
this story appeared on asia.nikkei.com . 

#(MONEY) #(MONEY) #(MONEY) 

suspects-arrested-san-jose-shooting-killed-1-18697321.php : 

san(GPE) jose(GPE) police said thursday(DATE) they arrested two(CARDINAL) men accused of shooting two(CARDINAL) other men , one(CARDINAL) fatally , in december(DATE) .

Source Document 3:
joshua(PERSON) sodapop(PERSON) bocanegra(GPE) , 20(DATE) , of san(GPE) jose(GPE) was arrested in san(GPE) francisco(GPE) on feb(DATE) .(DATE) 15(DATE) . that same day , police also arrested michael(PERSON) carabajal(PERSON) , 20(DATE) , of gilroy(ORG) , in gilroy(ORG) , police said . both suspects were booked into santa(GPE) clara(GPE) county(GPE) main jail on suspicion of homicide .

So